In [ ]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2

# Try 7

In [ ]:
paper_size = '7x5 inches'
border:float=10
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:
new, old = gp.split_poly(drawbox, gp.random_line_subdivide(drawbox, x0=0.87, x1=0.35))

bg_polys = [new] + gp.split_poly(old, gp.random_line_subdivide(old, x0=0.03, x1=0.69))

bg_right, bg_mid, bg_left = bg_polys
MultiPolygon(bg_polys)

In [ ]:
xgen = ss.uniform(loc=0.55, scale=0.1).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.99, 
    depth=0, 
    depth_limit=4,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])
bps

In [ ]:
bg_bps_left = [so.unary_union([bp for bp in bps if bp.within(bg_left)]).buffer(0.1)]
bg_bps_right = [so.unary_union([bp for bp in bps if bp.within(bg_right)]).buffer(0.1)]

In [ ]:
bg_bps_left = [bg_left]
bg_bps_right = [bg_right]
brush_left = [bg_left]

In [ ]:
bg_layers = []
for bg_bps in [bg_bps_left, bg_bps_right, brush_left]:
    fills = []
    for p in bg_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.5),
            degrees=np.random.uniform(10, 40),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=50,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    bg_layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(bg_layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
for tolerance in [0.3, 0.6, 1.2]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')
sk.display()

In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.6,
    rad_seq_end=0.08,
    min_allowed_rad=3,
    n_rads=10,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.6
rcp.rad_seq_end=0.08
rcp.n_rads=80
rcp.run(10, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.13,
    rad_seq_end=0.13, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.1, 2.).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
occluded_tris = tris.intersection(rcp.unfilled_circles).buffer(-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.18,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.1, 1.7).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.28,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris]).buffer(-1e-6)

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(10), rad_seq_start=0.15,
    rad_seq_end=0.15, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.2
trcp.rad_seq_end=0.2
trcp.run(4)

In [ ]:
length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(0.8, 1.4).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.38,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = gp.merge_Polygons([all_polys, occluded_tris])

In [ ]:

sk.stroke(4)
sk.geometry(layer1)
sk.stroke(5)
sk.geometry(layer2)
sk.stroke(6)
sk.geometry(layer3)
sk.display(color_mode='layer')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0284_cpack_beams.svg'

sk.save(savepath)

vpype_commands = 'reloop linesimplify --tolerance 0.1mm linemerge --tolerance 0.1mm linesort'
vpype_str = f'vpype read -q 0.05mm {savepath} {vpype_commands} write {savepath}'

os.system(vpype_str)

# Try 8

In [ ]:
paper_size = '7x5 inches'
border:float=10
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:
xgen = ss.uniform(loc=0.55, scale=0.1).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.8, 
    depth=0, 
    depth_limit=6,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])
bps

In [ ]:
bg_bps_left = [bps]

In [ ]:
bg_layers = []
for bg_bps in bg_bps_left:
    fills = []
    for p in bg_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.4, 0.7),
            degrees=np.random.uniform(10, 140),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=50,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    bg_layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(bg_layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
for tolerance in [0.3, 0.6, 1.2]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')
sk.display()

In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.6,
    rad_seq_end=0.08,
    min_allowed_rad=3,
    n_rads=10,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.6
rcp.rad_seq_end=0.08
rcp.n_rads=80
rcp.run(10, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.13,
    rad_seq_end=0.13, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.1, 2.).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
occluded_tris = tris.intersection(rcp.unfilled_circles).buffer(-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.14,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.1, 1.7).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.28,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris]).buffer(-1e-6)

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(10), rad_seq_start=0.15,
    rad_seq_end=0.15, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.2
trcp.rad_seq_end=0.2
trcp.run(4)

In [ ]:
length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(0.8, 1.4).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.38,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = gp.merge_Polygons([all_polys, occluded_tris])

In [ ]:
bg_layer = gp.robust_difference(bg_layers[0], all_polys)

In [ ]:
bg_layer

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.stroke(1)
sk.geometry(bg_layer)
sk.stroke(4)
sk.geometry(layer1)
sk.stroke(5)
sk.geometry(layer2)
sk.stroke(6)
sk.geometry(layer3)
sk.penWidth('0.3')
for tolerance in [0.1, 0.2, 0.3]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0287_cpack_beams.svg'

sk.save(savepath)


# Try 9

In [ ]:
paper_size = '7x5 inches'
border:float=10
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:
xgen = ss.uniform(loc=0.3, scale=0.25).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.8, 
    depth=0, 
    depth_limit=9,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])
bps

In [ ]:
bg_bps_left = [bps]

In [ ]:
bg_layers = []
for bg_bps in bg_bps_left:
    fills = []
    for p in bg_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.2, 1.)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.12, 0.6),
            degrees=np.random.uniform(10, 140),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=15,
            n_eval_points=30,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    bg_layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(bg_layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
# for tolerance in [0.3, 0.6, 1.2]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')
sk.display()

In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.6,
    rad_seq_end=0.08,
    min_allowed_rad=3,
    n_rads=10,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.6
rcp.rad_seq_end=0.08
rcp.n_rads=80
rcp.run(10, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.13,
    rad_seq_end=0.13, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.1, 2.).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
occluded_tris = tris.intersection(rcp.unfilled_circles).buffer(-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.19,angles=-45,)
stp.d_buffers += np.random.uniform(-0.06, 0.12, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.1, 1.7).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.28,angles=-45,)
stp.d_buffers += np.random.uniform(-0.08, 0.08, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris]).buffer(-1e-6)

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(10), rad_seq_start=0.15,
    rad_seq_end=0.15, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.2
trcp.rad_seq_end=0.2
trcp.run(1)

In [ ]:
length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(0.4, 0.8).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles)
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.38,angles=-45,)
stp.d_buffers += np.random.uniform(-0.09, 0.09, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = gp.merge_Polygons([all_polys, occluded_tris])

In [ ]:
bg_layer = gp.robust_difference(bg_layers[0], all_polys)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.stroke(1)
sk.geometry(bg_layer)
sk.stroke(1)
sk.geometry(layer1)
sk.stroke(1)
sk.geometry(layer2)
sk.stroke(1)
sk.geometry(layer3)
sk.penWidth('0.3')
sk.vpype('splitall')
# for tolerance in [0.2,]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm linesimplify --tolerance 0.1')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0293_cpack_beams.svg'

sk.save(savepath)


# Try 10

In [ ]:
paper_size = '7x5 inches'
border:float=10
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:
xgen = ss.uniform(loc=0.35, scale=0.25).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.8, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])
bps


In [ ]:
bg_bps_left = [bps]

In [ ]:
bg_layers = []
for bg_bps in bg_bps_left:
    fills = []
    for p in bg_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.15, 0.8)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.3, 0.45),
            degrees=np.random.uniform(10, 170),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.2,
            n_nodes_per_line=15,
            n_eval_points=40,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    bg_layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(bg_layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
# for tolerance in [0.3, 0.6, 1.2]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')
sk.display()

In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.55,
    rad_seq_end=0.09,
    min_allowed_rad=10,
    n_rads=20,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.4
rcp.rad_seq_end=0.1
rcp.n_rads=80
rcp.run(4, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.13,
    rad_seq_end=0.13, min_allowed_rad=4, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(0.8, 1.7).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
occluded_tris = tris.intersection(rcp.unfilled_circles)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.1, 1.7).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles).buffer(0.5)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.3,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris])

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(10), rad_seq_start=0.15,
    rad_seq_end=0.15, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.2
trcp.rad_seq_end=0.2
trcp.run(1)

In [ ]:
length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(0.4, 1.2).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles).buffer(2.3)
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.38,angles=-45,)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = gp.merge_Polygons([all_polys, occluded_tris])

In [ ]:
mbg = gp.merge_LineStrings([l for l in bg_layers[0] if l.length>0.05])

In [ ]:
bg_layer = gp.robust_difference(mbg, all_polys.buffer(1e-6))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.stroke(1)
sk.geometry(bg_layer)
for tolerance in [0.1, 0.2, 0.4, 0.8]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.display(color_mode='none')

In [ ]:
sk.stroke(2)
sk.geometry(layer1)
sk.stroke(3)
sk.geometry(layer2)
sk.stroke(4)
sk.geometry(layer3)
sk.penWidth('0.3')

# tolerance=0.1
# sk.vpype(f'linesimplify --tolerance {tolerance}')
# for tolerance in [0.1, 0.2, 0.4]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.display(color_mode='none')

In [ ]:
savepath = '/mnt/c/code/side/plotter_images/oned_outputs/0301_cpack_beams.svg'

sk.save(savepath)


# Try 11

In [ ]:
paper_size = '7x5 inches'
border:float=10
paper = utils.Paper(paper_size)

drawbox = paper.get_drawbox(border)
buffer_style = 2

In [ ]:
xgen = ss.uniform(loc=0.35, scale=0.25).rvs
split_func = functools.partial(gp.split_along_longest_side_of_min_rectangle, xgen=xgen)
splits = gp.recursive_split_frac_buffer(
    drawbox, 
    split_func=split_func,
    p_continue=0.8, 
    depth=0, 
    depth_limit=7,
    buffer_frac=-0.0
)

bps = MultiPolygon([p for p in splits])
bps


In [ ]:
bg_bps_left = [bps]

In [ ]:
bg_layers = []
for bg_bps in bg_bps_left:
    fills = []
    for p in bg_bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.15, 0.8)).rvs
        bhf = gp.BezierHatchFill(
            spacing=np.random.uniform(0.3, 0.45),
            degrees=np.random.uniform(10, 170),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.2,
            n_nodes_per_line=15,
            n_eval_points=40,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    bg_layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(bg_layers):
    sk.stroke(i+1)
    sk.geometry(layer)

sk.penWidth('0.3')
# for tolerance in [0.3, 0.6, 1.2]:
#     sk.vpype(f'linemerge --tolerance {tolerance}mm')
# sk.vpype('linesimplify --tolerance 0.1 linesort')
sk.display()

In [ ]:
# init
rcp = gp.RecursiveCirclePacker(
    drawbox, 
    rad_seq_start=0.6,
    rad_seq_end=0.09,
    min_allowed_rad=10,
    n_rads=20,
)
rcp.run(1)

In [ ]:
rcp.rad_seq_start=0.55
rcp.rad_seq_end=0.1
rcp.n_rads=80
rcp.run(4, progress_bar=True)
rcp.unfilled_circles

## lattice 1

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.13,
    rad_seq_end=0.13, min_allowed_rad=4, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.29
trcp.rad_seq_end=0.18
trcp.run(3)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(0.8, 1.7).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
occluded_tris = tris.intersection(rcp.unfilled_circles)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.2,angles=-45, d_translate_factor=0.65)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer1 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = occluded_tris

## lattice 2

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(20), rad_seq_start=0.16,
    rad_seq_end=0.16, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.3
trcp.rad_seq_end=0.3
trcp.run(4)

length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(1.1, 1.7).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)
tris

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles).buffer(0.5)

In [ ]:
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.25,angles=-45,d_translate_factor=0.65)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer2 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = so.unary_union([all_polys, occluded_tris])

## lattice3

In [ ]:
trcp = gp.RecursiveCirclePacker(drawbox.buffer(10), rad_seq_start=0.15,
    rad_seq_end=0.15, min_allowed_rad=2, n_rads=10, )
trcp.run(1)
trcp.min_allowed_rad=5
trcp.rad_seq_start=0.2
trcp.rad_seq_end=0.2
trcp.run(1)

In [ ]:
length_filter_gen = ss.uniform(96, 185).rvs
buffer_size_gen = ss.uniform(0.4, 1.2).rvs
tri_vertices = MultiPoint([c.centroid for c in trcp.unfilled_circles])
trils = MultiLineString(so.triangulate(tri_vertices, edges=True))

filt_trils = MultiLineString([l for l in trils if l.length<length_filter_gen()])
tris = [f.buffer(buffer_size_gen(), cap_style=2, join_style=3) for f in filt_trils]
tris = so.unary_union(tris)

In [ ]:
int_tris = tris.intersection(rcp.unfilled_circles).buffer(5.3)
occluded_tris = gp.robust_difference(int_tris, all_polys, buffer_distance=-1e-6)

In [ ]:
stp = gp.ScaleTransPrms(d_buffer=-0.3,angles=-45,d_translate_factor=0.65)
stp.d_buffers += np.random.uniform(-0.03, 0.03, size=stp.d_buffers.shape)

In [ ]:
fills = []
for p in occluded_tris:
    P = gp.Poly(p)
    P.fill_scale_trans(**stp.prms)
    fills.append(P.fill)

In [ ]:
layer3 = gp.merge_LineStrings(fills)

In [ ]:
all_polys = gp.merge_Polygons([all_polys, occluded_tris])

In [ ]:
mbg = gp.merge_LineStrings([l for l in bg_layers[0] if l.length>0.05])

In [ ]:
bg_layer = gp.robust_difference(mbg, all_polys.buffer(1e-6))

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.stroke(1)
sk.geometry(bg_layer)
for tolerance in [0.1, 0.2, 0.4, 0.8]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.display(color_mode='none')

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
sk.stroke(2)
sk.geometry(layer1)
sk.stroke(3)
sk.geometry(layer2)
sk.stroke(4)
sk.geometry(layer3)
sk.penWidth('0.3')

tolerance=0.1
sk.vpype(f'linesimplify --tolerance {tolerance}')
for tolerance in [0.1, 0.2, 0.4]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesort')
sk.display(color_mode='none')

In [ ]:
savepath = '/Users/naka/code/side/plotter_images/oned_outputs/0315_cpack_beams_postcard.svg'

sk.save(savepath)
